In [7]:
import os
len(os.listdir("/scratch/network/mk8574/change_detection_challenge/swapdata/train/x")) #877

1101

In [5]:
import pandas as pd
train_csv= pd.read_csv('/scratch/network/mk8574/change_detection_challenge/data/train.csv')
lst = []
for i in range(12000):
    if train_csv["id"][i] not in os.listdir("/scratch/network/mk8574/change_detection_challenge/swapdata/train/x") and ((train_csv["class1"][i]!=0 and train_csv["class2"][i]==0) or(train_csv["class1"][i]==0 and train_csv["class2"][i]!=0)) and train_csv["class3"][i]==0:
        lst.append(train_csv["id"][i])
print(len(lst))

429


In [3]:
f  = open("/scratch/network/mk8574/change_detection_challenge/baseline_youhan/results/train/20231118_071639/valids","r")
val_lst = f.readline()
print(val_lst[0])

[


In [4]:
import ast
val_lst = ast.literal_eval(val_lst)
print(val_lst[0])

../data/train/x/2017_EPG_2LB_000823.png


In [49]:
import pandas as pd
a = [0,0,0]
train_csv= pd.read_csv('/scratch/network/mk8574/change_detection_challenge/data/train.csv')

for i in val_lst:
    val_df = train_csv[train_csv["id"]==i[16:]]
   
    if(int(val_df["class1"])):a[0]+=1
    if(int(val_df["class2"])):a[1]+=1
    if(int(val_df["class3"])) :a[2] +=1
print(a)
print(list(map(lambda x:round(x/sum(a)*100,2),a)))

/tmp/ipykernel_992066/1814041015.py:8: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  if(int(val_df["class1"])):a[0]+=1
/tmp/ipykernel_992066/1814041015.py:9: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  if(int(val_df["class2"])):a[1]+=1
/tmp/ipykernel_992066/1814041015.py:10: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  if(int(val_df["class3"])) :a[2] +=1


[307, 156, 2129]
[11.84, 6.02, 82.14]


In [22]:
print(a)

[1935   NaN
Name: class1, dtype: float64, 0, 0]


In [35]:
print(16*600)

9600


In [48]:
"""Train
"""
from datetime import datetime
from time import time
import numpy as np
import shutil, random, os, sys, torch
from glob import glob
from torch.utils.data import DataLoader, ConcatDataset
from sklearn.model_selection import train_test_split

prj_dir = os.path.dirname("/scratch/network/mk8574/change_detection_challenge/baseline_dy/")
sys.path.append(prj_dir)

#checkpoint_path = 'results/train/20231117_214934/model.pt'
checkpoint_path = None

from modules.utils import load_yaml, get_logger
from modules.metrics import get_metric_function
from modules.earlystoppers import EarlyStopper
from modules.losses import get_loss_function
from modules.optimizers import get_optimizer
from modules.schedulers import get_scheduler
from modules.scalers import get_image_scaler
from modules.datasets import SegDataset
from modules.recorders import Recorder
from modules.trainer import Trainer
from models.utils import get_model

if __name__ == '__main__':
    
    # Load config
    config_path = os.path.join(prj_dir, 'config', 'train.yaml')
    config = load_yaml(config_path)
    
    # Set train serial: ex) 20211004
    train_serial = datetime.now().strftime("%Y%m%d_%H%M%S")
    train_serial = 'debug' if config['debug'] else train_serial

    # Set random seed, deterministic
    torch.cuda.manual_seed(config['seed'])
    torch.manual_seed(config['seed'])
    np.random.seed(config['seed'])
    random.seed(config['seed'])
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # Set device(GPU/CPU)
    os.environ['CUDA_VISIBLE_DEVICES'] = str(config['gpu_num'])
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Create train result directory and set logger
    train_result_dir = os.path.join(prj_dir, 'results', 'train', train_serial)
    os.makedirs(train_result_dir, exist_ok=True)

    # Set logger
    logging_level = 'debug' if config['verbose'] else 'info'
    logger = get_logger(name='train',
                        file_path=os.path.join(train_result_dir, 'train.log'),
                        level=logging_level)


    # Set data directory
    train_dirs = os.path.join('/scratch/network/mk8574/change_detection_challenge/data', 'train')

    # Load data and create dataset for train 
    # Load image scaler
    train_img_paths = glob(os.path.join(train_dirs, 'x', '*.png'))
    train_img_paths, val_img_paths = train_test_split(train_img_paths, test_size=config['val_size'], random_state=config['seed'], shuffle=True)

    train_dataset = SegDataset(paths=train_img_paths,
                            input_size=[config['input_width'], config['input_height']],
                            scaler=get_image_scaler(config['scaler']),
                            logger=logger)
    
    #swapped img 
    train_dirs_swap = os.path.join('/scratch/network/mk8574/change_detection_challenge/swapdata', 'train')
    print(train_dirs_swap)
    print(train_dirs)
    print(len(train_dataset))
    train_img_paths_swap = glob(os.path.join(train_dirs_swap, 'x', '*.png'))
    print(os.path.join(train_dirs_swap, 'x', '*.png'))
    print(os.path.join(train_dirs, 'x', '*.png'))
    train_dataset_swap = SegDataset(paths=train_img_paths_swap,
                            input_size=[config['input_width'], config['input_height']],
                            scaler=get_image_scaler(config['scaler']),
                            logger=logger)
    print(len(train_dataset_swap))
    train_dataset = ConcatDataset([train_dataset, train_dataset_swap])

/scratch/network/mk8574/change_detection_challenge/swapdata/train
/scratch/network/mk8574/change_detection_challenge/data/train
9600
/scratch/network/mk8574/change_detection_challenge/swapdata/train/x/*.png
/scratch/network/mk8574/change_detection_challenge/data/train/x/*.png
1235
